# Step1: get code from github

In [ ]:
from github import Github
from github import Auth
import re
import tqdm
import time
import random

In [2]:
repo_url="rust-lang/rust"
token=""# your github token here

In [ ]:
# get repo
def get_repo(repo_url,token):
    auth=Auth.Token(token)
    g=Github(auth=auth)
    repo=g.get_repo(repo_url)
    return repo

In [ ]:
# get issue
def get_issues(repo_url,token,state):
    auth=Auth.Token(token)
    g=Github(auth=auth)
    repo=g.get_repo(repo_url)
    issues=repo.get_issues(state=state)
    return issues

In [ ]:
closed_issues=get_issues(repo_url,token,state="closed")

In [ ]:

print(closed_issues.totalCount)

In [ ]:

def get_rust_code_from_issue(issue,need_split=False):
    pattern = re.compile(r'```[Rr][Uu][Ss][Tt][\s\S]*?```|```[Rr][Ss][\s\S]*?```', re.MULTILINE)
    
    codes_from_issue_body=[]
    try:
        res=pattern.search(issue.body)
    except TypeError:
        res=None
    if res is not None:
        codes_from_issue_body+=[res.group()]
    comments=issue.get_comments()
    # deal with rate limit
    time.sleep(random.uniform(1, 1.5))
    codes_from_comments=[]
    for comment in comments:
        
        try:
            res=pattern.search(comment.body)
        except TypeError:
            res=None
        if res is not None:
            codes_from_comments+=[res.group()]
    if need_split:
        return codes_from_issue_body,codes_from_comments
    else:
        return codes_from_issue_body+codes_from_comments


In [ ]:
# deel with raw rust code snippets
def deel_raw_rust_code(code):
    code_lines=code.splitlines()
    if len(code_lines)<=2:
        return ""
    else:
        return "\n".join(code_lines[1:-1])
# write code to .rs file in rust_code folder
def save_rust_code_to_file(code,issue_num,cnt,folder_path):
    file_path=folder_path+"/"+str(issue_num)+"_"+str(cnt)+".rs"
    with open(file_path,"w") as f:
        f.write(code)

In [ ]:
folder_path='./database/issue_code'
for close_issue in closed_issues:
    issue_num=close_issue.number
    if(issue_num>59094):
        continue
    codes=get_rust_code_from_issue(close_issue)
    
    cnt=0
    for code in codes:
        cnt+=1
        code=deel_raw_rust_code(code)
        save_rust_code_to_file(code,issue_num,cnt,folder_path)

    

In [ ]:
repo=get_repo(repo_url,token)
issue=repo.get_issue(number=55094)
code1,code2=get_rust_code_from_issue(issue,True)


In [ ]:
len(code2)

In [ ]:
code=deel_raw_rust_code(code2[0])
print(code)

In [ ]:
codes=code1+code2
for code in codes:
    code=deel_raw_rust_code(code)
    

In [ ]:
import requests

url = 'https://api.github.com/rate_limit'
headers = {'Authorization': token}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    limit = data['rate']['limit']  
    remaining = data['rate']['remaining']  
    reset_time = data['rate']['reset']  
    

In [ ]:
# deal code: delete none or duplicate code
def deal_rust_files(issue_num):
    pass

In [ ]:

import hashlib
import os

def get_file_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def get_file_hash(content):
    sha1_hash = hashlib.sha1(content.encode()).hexdigest()
    return sha1_hash

def find_duplicate_files(directory):
    file_hashes = {}
    duplicate_files = []

    for filename in os.listdir(directory):
        if filename.endswith('.rs'):
            file_path = os.path.join(directory, filename)
            content = get_file_content(file_path)
            file_hash = get_file_hash(content)

            if file_hash in file_hashes:
                duplicate_files.append(file_path)
            else:
                file_hashes[file_hash] = file_path

    return duplicate_files


directory = '/path/to/directory'

duplicate_files = find_duplicate_files(directory)

if duplicate_files:
    print("repeat files:")
    for file in duplicate_files:
        print(file)
else:
    print("ok")